In [ ]:
!gdown --id 1ZGIHE4DF1Nb316jPBWyU7Zzp_K1Lp5xM

In [ ]:
!unzip "/content/TaskA-Memes.zip"
!rm TaskA-Memes.zip

In [ ]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
image_path = '/content/TaskA-Memes/'

In [ ]:
batch_size = 16
epochs = 50
image_size = 224

In [ ]:
def loadImage(image_path): # /content/MAMI/TRAINING
    try:
        return load_img(image_path, target_size=(image_size, image_size))
    except:
        image_path = image_path.replace('png', 'jpg')
        return load_img(image_path, target_size=(image_size, image_size))

In [ ]:
train_df = pd.read_csv('/content/TaskA-Memes/task_a.csv')

In [ ]:
train_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
train_df = train_df[train_df.meme_type != 'only offensive']

In [ ]:
train_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df['hate_speech'] = [1 if meme == 'hate_speech' else 0 for meme in train_df.meme_type]
train_df.drop(columns='meme_type', inplace=True)

In [ ]:
train_df

,file_name,text,hate_speech
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,1
2,img10.jpg,لا أحد النسويات,1
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,1
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,1
...,...,...,...
1669,IMG_269.jpg,لاتوجد نساء اجمل منك مهما بحثت والله؟! يعني بح...,0
1670,IMG_233.jpg,أنا أخوض حرب لا أعلم عنها شيئا,0
1671,IMG_138.jpg,لما اروح عند دكتور الاسنان يادكتور بالله عليك ...,0
1672,IMG_705.jpg,الضربة اللي متموتش تقوي لأ انا عايزاللي تموت,0


In [ ]:
path = image_path+'/'

train_df['image_path'] = path + train_df['file_name']

In [ ]:
train_df['image'] = None
train_df['image'] = train_df['image_path'].apply(lambda x: img_to_array(loadImage(x))) 

In [ ]:
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=41)

In [ ]:
X_train = train_df[['file_name', 'image']]
y_train = train_df['hate_speech']

X_test = test_df[['file_name', 'image']]
y_test = test_df['hate_speech']

In [ ]:
tmp = []
for value in X_train['image']:
  tmp.append(value) 
iX_train = np.array(tmp)

In [ ]:
tmp = []
for value in X_test['image']:
  tmp.append(value) 
iX_test = np.array(tmp)

In [ ]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
model_path = '/content/Best_model'

# earlystopping_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

In [ ]:
#_______________________________IMAGE MODEL_______________________________
l2_strength = 1e-5

input_image = layers.Input(shape=(image_size,image_size,3))
model = EfficientNetV2L(input_tensor=input_image, weights='imagenet', include_top=False)

for layer in model.layers:
    layer.trainable = False

x = model.output
x = layers.Flatten(input_shape=model.output_shape[1:])(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l2_strength))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_strength))(x)
x = layers.Dense(2, activation='softmax')(x)

image_model = Model(model.input, x)

image_model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
# image_model.summary()

history = image_model.fit(iX_train, 
                    y_train,
                    validation_data=(iX_test, y_test), # Remove
                    epochs= epochs,
                    batch_size=batch_size,
                    callbacks=[checkpoint_callback]
)   

In [ ]:
model_output = image_model.predict(iX_test) 

In [ ]:
preds = []
for out in model_output:
  preds.append(np.argmax(out))

In [ ]:
labels = []
for x in y_test:
  labels.append(np.argmax(x))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print("F1 macro:   {}".format(round(f1_score(labels, preds, average="macro"), 3)))
print("Precision:   {}".format(precision_score(labels,preds, average="macro"), 3))
print("Recall:   {}".format(recall_score(labels,preds, average="macro"), 3))
print("F1 micro:   {}".format(round(f1_score(labels, preds, average="micro"), 3)))